<a href="https://colab.research.google.com/github/sibteali786/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code/blob/main/18_fasttext_classification/fasttext_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>NLP Tutorial: Text Classification Using FastText</h3>

##### Dataset Credits: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

We have a dataset of ecommerce item description. Total 4 categories,
1. Household
2. Electronics
3. Clothing and Accessories
4. Books

The task at hand is to classify a product into one of the above 4 categories based on the product description

In [1]:
import pandas as pd
import requests
import io
url = "https://raw.githubusercontent.com/codebasics/nlp-tutorials/refs/heads/main/18_fasttext_classification/ecommerce_dataset.csv"
eccomerce_data = io.StringIO(requests.get(url).text)

df= pd.read_csv(eccomerce_data, names=["category", "description"], header=None)
print(df.shape)
df.head(3)

(50425, 2)


,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


**Drop NA values**

In [2]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [3]:
df.category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [4]:
df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

<ipython-input-4-5cbb1445e79b>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)


In [5]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

When you train a fasttext model, it expects labels to be specified with __label__ prefix. We will just create a third column in the dataframe that has __label__ as well as the product description

In [6]:
df['category'] = '__label__' + df['category'].astype(str)
df.head(5)

,category,description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [7]:
df['category_description'] = df['category'] + ' ' + df['description']
df.head(3)

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


**Pre-procesing**
1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case

In [10]:
import re

text = "  VIKI's | Bookcase/Bookshelf (3-Shelf/Shelve, White) | ? . hi"
text = re.sub(r'[^\w\s\']', ' ', text)
text = re.sub(' +', ' ', text)
text.lower()

" viki's bookcase bookshelf 3 shelf shelve white hi"

In [11]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [12]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


**Train Test Split**

In [14]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [15]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [16]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

**Train the model and evaluate performance**

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)


In [17]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

ModuleNotFoundError: No module named 'fasttext'

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

**Now let's do prediction for few product descriptions**

In [ ]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99791646]))

In [ ]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [ ]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000966]))

In [ ]:
model.get_nearest_neighbors("painting")

[(0.9961369037628174, 'vacuum'),
 (0.9959887862205505, 'guard'),
 (0.9959416389465332, 'alarm'),
 (0.9958606958389282, 'lint'),
 (0.9955978989601135, 'temperature'),
 (0.995332658290863, 'machine'),
 (0.9952465295791626, 'gloss'),
 (0.9952084422111511, 'door'),
 (0.9951741099357605, 'steam'),
 (0.9947825074195862, 'induction')]

In [ ]:
model.get_nearest_neighbors("sony")

[(0.9984563589096069, 'tablets'),
 (0.9980253577232361, 'dvd'),
 (0.9979312419891357, 'binocular'),
 (0.9977967739105225, 'colour'),
 (0.9977490305900574, 'external'),
 (0.9976783394813538, 'player'),
 (0.9972286820411682, 'viewing'),
 (0.9971092939376831, 'photos'),
 (0.9967519640922546, 'binoculars'),
 (0.996209979057312, 'graphics')]

In [ ]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'spaulding'),
 (0.0, 'audette'),
 (0.0, 'rheumatologist')]